In [1]:
# Step 1: Import necessary libraries
import numpy as np
import os
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile
from io import BytesIO
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import InceptionV3


In [2]:
# Define the URL of the dataset
url = 'https://github.com/SriPenumatcha/Flower-Identification-using-deep-learning/archive/refs/heads/main.zip'

# Download the dataset
response = requests.get(url)
if response.status_code == 200:
    # Extract the dataset
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('/tmp')

# Path to the extracted dataset
data_dir = '/tmp/Flower-Identification-using-deep-learning-main/CleanedDataSet'

# Verify paths and dataset structure
folders = os.listdir(data_dir)
print("Folders:", folders)

# Step 3: Load and prepare the dataset
data = load_files(data_dir)
X = np.array(data['filenames'])
y = np.array(data['target'])

# Function to convert images to arrays
def convert_img_to_arr(file_path_list):
    arr = []
    img_width, img_height = 150, 150
    for file_path in file_path_list:
        img = load_img(file_path, target_size=(img_width, img_height))
        img = img_to_array(img)
        arr.append(img)
    return arr

# Convert images to arrays
X = np.array(convert_img_to_arr(X))
print(X.shape)
print('First training item:', X[0])



Folders: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
(4290, 150, 150, 3)
First training item: [[[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  1.   1.   3.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 [[  0.   0.   0.]
  [  1.   1.   1.]
  [  1.   1.   0.]
  ...
  [  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]]

 ...

 [[ 68. 111.  40.]
  [ 38.  92.  34.]
  [ 10.  33.   4.]
  ...
  [ 75.  92.  37.]
  [ 32.  48.   9.]
  [ 12.  27.   6.]]

 [[ 44. 105.  35.]
  [ 17.  56.  11.]
  [  9.  32.  16.]
  ...
  [ 21.  34.   8.]
  [ 62.  86.  28.]
  [ 35.  45.  20.]]

 [[ 29.  84.  19.]
  [  4.  20.   9.]
  [ 54.  94.  44.]
  ...
  [ 19.  36.   2.]
  [ 65.  84.  28.]
  [ 27.  46.  16.]]]


In [3]:
# Create an ImageDataGenerator with rescaling and validation split
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2)

# Prepare training and validation generators
train_generator = datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')
validation_generator = datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(folders), activation='softmax')
])

# Print model layers
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=2, validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


Found 3433 images belonging to 5 classes.
Found 857 images belonging to 5 classes.


C:\Users\sri_p\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    44,302,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,398,661 (169.37 MB)

 Trainable params: 44,398,661 (169.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2


C:\Users\sri_p\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.3377 - loss: 2.0696 - val_accuracy: 0.5624 - val_loss: 1.0792
Epoch 2/2
108/108 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.5798 - loss: 1.0634 - val_accuracy: 0.5881 - val_loss: 1.0094
27/27 ━━━━━━━━━━━━━━━━━━━━ 10s 357ms/step - accuracy: 0.5861 - loss: 1.0268
Test accuracy: 0.5880979895591736


In [4]:
# Use a pretrained VGG16 model
datagen = ImageDataGenerator(
    rescale=1/255, 
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Prepare training and validation generators
train_generator = datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=8, class_mode='categorical', subset='training')
validation_generator = datagen.flow_from_directory(data_dir, target_size=(224, 224), batch_size=8, class_mode='categorical', subset='validation')

model_vgg16 = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

model_vgg16.summary()

for layer in model_vgg16.layers:
    layer.trainable = False

model = Sequential()
model.add(model_vgg16)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(folders), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_generator, epochs=3, validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')



Found 3433 images belonging to 5 classes.
Found 857 images belonging to 5 classes.


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 1088s 3s/step - accuracy: 0.5896 - loss: 2.0475 - val_accuracy: 0.7701 - val_loss: 0.6223
Epoch 2/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 1108s 3s/step - accuracy: 0.7713 - loss: 0.6114 - val_accuracy: 0.7911 - val_loss: 0.5658
Epoch 3/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 1051s 2s/step - accuracy: 0.8385 - loss: 0.4540 - val_accuracy: 0.7736 - val_loss: 0.6183
108/108 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.7551 - loss: 0.6612
Test accuracy: 0.7701283693313599


In [5]:
# Function to train a model
def train_model(new_model):
    # Freeze layers
    for layer in new_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(new_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(len(folders), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(train_generator, epochs=3, validation_data=validation_generator)
    return model

new_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
model = train_model(new_model)

# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
Epoch 1/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 379s 855ms/step - accuracy: 0.6394 - loss: 10.6763 - val_accuracy: 0.7911 - val_loss: 0.5740
Epoch 2/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 355s 823ms/step - accuracy: 0.8269 - loss: 0.5129 - val_accuracy: 0.8460 - val_loss: 0.4753
Epoch 3/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 411s 955ms/step - accuracy: 0.8573 - loss: 0.4079 - val_accuracy: 0.8273 - val_loss: 0.5362
108/108 ━━━━━━━━━━━━━━━━━━━━ 58s 537ms/step - accuracy: 0.8399 - loss: 0.5225
Test accuracy: 0.8401400446891785


In [6]:
# Function to load and preprocess the image
def load_and_preprocess_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# URL of the image to predict
image_url = 'https://www.thetutuguru.com.au/wp-content/uploads/2020/07/Black-Madonna-Rose.jpg'

# Load and preprocess the image
img_array = load_and_preprocess_image(image_url)

# Predict the class of the image
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)

# Get the class label
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
predicted_label = class_labels[predicted_class[0]]

print(f'The predicted class is: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
The predicted class is: rose
